2017.7.17


In [1]:
import pandas as pd
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
import xgboost as xgb
import time

In [3]:
train_data_set = pd.read_csv("feature/xgb_train_data.csv")
test_data_set = pd.read_csv("feature/xgb_test_data.csv")


In [4]:
# 合并 训练数据 和 测试 数据，进行统一处理
test_data_set['flag'] = -999
train_test_data = pd.concat([train_data_set, test_data_set], axis=0)

In [28]:
train_test_data.head(5)

,age,education_num,flag,hours_per_week,income,marital_status,race,relationship,sex,workclass
0,39,13,NaN,1,0,2,1,2,1,4
1,50,13,NaN,0,0,1,1,1,1,2
2,38,9,NaN,1,0,3,1,2,1,1
3,53,7,NaN,1,0,1,2,1,1,1
4,28,13,NaN,1,0,1,2,5,0,1


In [31]:
# 合并 训练数据 和 测试 数据，进行统一处理
test_data_set['flag'] = -999
train_test_data = pd.concat([train_data_set, test_data_set], axis=0)

category_val = ["education_num", "marital_status", "relationship", "workclass"]
for val in category_val:
    val_dummies = pd.get_dummies(train_test_data[val])
    val_dummies.columns = [val + '_' + str(i) for i in range(val_dummies.shape[1])]
    train_test_data.drop(val, axis=1, inplace=True)
    train_test_data = pd.concat([train_test_data, val_dummies], axis=1)

train_data = train_test_data[train_test_data.flag != -999]
test_data = train_test_data[train_test_data.flag == -999]

train_data.drop("flag", axis=1, inplace=True)
test_data.drop("flag", axis=1, inplace=True)

train_data.to_csv("feature/feature_train.csv", index=None)
test_data.to_csv("feature/feature_test.csv", index=None)

F:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
F:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [32]:
print(train_data.shape)
print(test_data.shape)

(32561, 43)
(16281, 43)


In [33]:
train_data.head(3)

,age,hours_per_week,income,race,sex,education_num_0,education_num_1,education_num_2,education_num_3,education_num_4,...,relationship_5,workclass_0,workclass_1,workclass_2,workclass_3,workclass_4,workclass_5,workclass_6,workclass_7,workclass_8
0,39,1,0,1,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,50,0,0,1,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,38,1,0,1,1,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
model_train_data = pd.read_csv("feature/feature_train.csv")
model_test_data = pd.read_csv("feature/feature_test.csv")

train_label = model_train_data["income"]
train_x = model_train_data.drop("income", axis=1)

test_label = model_test_data["income"]
test_x = model_test_data.drop("income", axis=1)

In [3]:
offset = round(0.7*train_x.shape[0])
print("offset is ", str(offset))

dtrain = xgb.DMatrix(train_x[:offset], label=train_label[:offset])
dval = xgb.DMatrix(train_x[offset:], label=train_label[offset:])

dtest = xgb.DMatrix(test_x)

watchlist = [(dtrain, 'train')]

#watchlist = [(dtrain, 'train'), (dval, 'val')]

params = {'booster': 'gbtree',
          'objective': 'binary:logistic',
          'eta': 0.001,
          'lambda': 30,
          'gamma': 3,
          'eval_metric': 'auc',
          'max_depth': 6,
          'subsample': 0.6,
          'colsample_bytree': 0.6,
          'min_child_weight': 3,
          'seed': 1024,
          'nthread': 8,
         }

#param_list = list(params.items())
num_round = 259




offset is  22793


In [ ]:
model = xgb.train(params, dtrain, num_boost_round=num_round, evals=watchlist, early_stopping_rounds=100)
model.save_model('model/xgb.model')
print("best best_ntree_limit", model.best_ntree_limit)

[0]	train-auc:0.804695
Will train until train-auc hasn't improved in 100 rounds.
[1]	train-auc:0.823594
[2]	train-auc:0.8299
[3]	train-auc:0.838425
[4]	train-auc:0.838612
[5]	train-auc:0.837589
[6]	train-auc:0.841515
[7]	train-auc:0.843114
[8]	train-auc:0.841324
[9]	train-auc:0.843074
[10]	train-auc:0.85312
[11]	train-auc:0.85347
[12]	train-auc:0.853047
[13]	train-auc:0.85323
[14]	train-auc:0.853464
[15]	train-auc:0.853648
[16]	train-auc:0.853386
[17]	train-auc:0.853711
[18]	train-auc:0.85377
[19]	train-auc:0.853715
[20]	train-auc:0.853541
[21]	train-auc:0.853442
[22]	train-auc:0.853523
[23]	train-auc:0.853249
[24]	train-auc:0.853094
[25]	train-auc:0.85313
[26]	train-auc:0.853246
[27]	train-auc:0.853254
[28]	train-auc:0.853166
[29]	train-auc:0.853255
[30]	train-auc:0.852691
[31]	train-auc:0.852736
[32]	train-auc:0.852726
[33]	train-auc:0.858756
[34]	train-auc:0.858432
[35]	train-auc:0.858387
[36]	train-auc:0.858365
[37]	train-auc:0.858372
[38]	train-auc:0.858889
[39]	train-auc:0.858798